In [77]:
import numpy as np # type: ignore

First helper function: linear interpolation

In [78]:
def linear_interpolation(x, y, x1):
    n = len(x)
    for i in range(n-1):
        if x[i] <= x1 <= x[i+1]:
            return ((x1 - x[i-1]) * y[i] + (x[i] - x1) * y[i-1]) / (x[i] - x[i-1])

Second helper function: Lagrange 4-point interpolation

In [79]:
def lagrange4_interpolation(x, y, x1):
    n = len(x)

    # take care of the case in which x1 is out of boundaries (because we need 2 extra points)
    if x1 < x[1]:
        return y[1]
    elif x1 > x[n-2]:
        return y[n-2]
    else:
        # "brute force" search (assuming small table)
        for i in range(1, n-1):
            if x[i] <= x1 <= x[i+1]:
                break

    # Ensure indices are within bounds
    i = max(2, min(i, n-2))

    x_points = x[i-2:i+2]
    y_points = y[i-2:i+2]

    # helper function to build the polinomes
    def polinome(k, x_val):
        term = 1
        for j in range(4):
            if j != k:
                term *= (x_val - x_points[j]) / (x_points[k] - x_points[j])
        return term

    P = 0
    for k in range(4):
        P += y_points[k] * polinome(k, x1)
    return P

Third helper function: Hermit 4-point interpolation

In [80]:
def hermit4_interpolation(x, y, x1):
    n = len(x)

    # take care of the case in which x1 is out of boundaries (because we need 2 extra points)
    if x1 < x[1]:
        return y[1]
    elif x1 > x[n-2]:
        return y[n-2]
    else:
        # "brute force" search (assuming small table)
        for i in range(1, n-1):
            if x[i] <= x1 <= x[i+1]:
                break

    # Ensure indices are within bounds
    i = max(1, min(i, n-2))

    # calculate the derivatives
    dy = np.zeros(n)
    for j in range(1, n-1):
        hf = x[j+1] - x[j]
        hb = x[j] - x[j-1]
        dy[j] = (y[j+1] / (2 * hf)) + y[j] * \
            ((hf - hb) / (2 * hf * hb)) - (y[j-1] / (2 * hb))

    x0_, x1_ = x[i], x[i+1]
    y0_, y1_ = y[i], y[i+1]
    dy0_, dy1_ = dy[i], dy[i+1]

    # Basis functions for Hermite interpolation
    def g1(x):
        return ((x - x0_) * (x - x1_)**2) / (x0_ - x1_)**2

    def g2(x):
        return ((x - x1_) * (x - x0_)**2) / (x1_ - x0_)**2

    def f1(x):
        return ((x - x1_)**2 / (x0_ - x1_)**2) - (2 * g1(x) / (x0_ - x1_))

    def f2(x):
        return ((x - x0_)**2 / (x1_ - x0_)**2) - (2 * g2(x) / (x1_ - x0_))

    # Hermite interpolation formula
    interpolation = (g1(x1) * dy0_ + f1(x1) * y0_ +
                     g2(x1) * dy1_ + f2(x1) * y1_)

    return interpolation

Partial derivation of the formula for 3-point derivative with unevenly spacede intervals: 
<div>
<img src="IMG_1978.jpg" width="1000"/>
</div>


### The main function: 

In [81]:
def interp(x, y, x1, itype):
    # x - an ordered one-dimensional numpy array of size n
    # y - is one-dimensional numpy array of the same size
    # x1 - a number larger than x[0] and smaller than x[n-1]
    # itype - either 'Linear', 'Lagrange4' or 'Hermit4'
    # The function returns the "y" value of x1 according to the requested type of interpolation
    if itype == 'Linear':
        return linear_interpolation(x, y, x1)
    elif itype == 'Lagrange4':
        return lagrange4_interpolation(x, y, x1)
    elif itype == 'Hermit4':
        return hermit4_interpolation(x, y, x1)
    else:
        raise ValueError(
            "Invalid interpolation type. Choose 'Linear', 'Lagrange4', or 'Hermit4'.")

We will use the following lines to initialize the arrays 
(x is not equally spaced 1-dimensional array starting from 0 to 100. It's the sum of two $1D$ arrays, the first is equally spaced from 1 to 99 with difference of 1  and the second consists of 99 random numbers in the range of $-1/3$ to $+1/3$):

In [82]:
npoints = 101
xmax = 10.
np.random.seed(12345)
x = np.arange(npoints,dtype="float64")
x[npoints-1] = xmax
r = np.random.rand(npoints-2)
x[1:npoints-1] = (x[1:npoints-1] + (r[0:npoints-2]-0.5)/1.5) * xmax/(npoints-1)
y=np.sin(x)
ntests = 10
x_test = np.random.rand(ntests) * xmax

x_test is one-dimensional numpy array of size 10 that holds test points for the interpolations.
Lets compare 'Linear', 'Lagrange4' or 'Hermit4' interpolations using commans like the following, which are given for linear interpolation:

In [83]:
y_interp_linear = np.zeros(ntests)
y_interp_lagrange = np.zeros(ntests)
y_interp_hermit = np.zeros(ntests)

for i in range(ntests):
    y_interp_linear[i] = interp(x, y, x_test[i], 'Linear')
    y_interp_lagrange[i] = interp(x, y, x_test[i], 'Lagrange4')
    y_interp_hermit[i] = interp(x, y, x_test[i], 'Hermit4')

Lets use the following function for printout of each type of interpolation:

In [84]:
def print_out(xv,yv):
    print('%18s %12.4e' % ('max. error was:',np.amax(np.abs(yv-np.sin(xv)))))
    print('%12s %12s %12s' % ('test_x', 'y_interp', 'error'))
    for i in range(ntests):
        print('%12.4e %12.4e %12.4e' % (xv[i], yv[i], yv[i]-np.sin(xv[i])))
    return

---

Result for linear interpolation: 

In [85]:
print_out(x_test, y_interp_linear)

   max. error was:   4.7076e-03
      test_x     y_interp        error
  4.2715e-01   4.1551e-01   1.2295e-03
  1.9579e+00   9.2768e-01   1.6958e-03
  2.9450e+00   1.9574e-01   4.2134e-04
  6.2700e+00  -1.3915e-02  -7.2873e-04
  8.6223e-01   7.6175e-01   2.4509e-03
  1.4295e+00   9.9444e-01   4.4114e-03
  5.1583e+00  -9.0694e-01  -4.7076e-03
  6.8934e+00   5.7689e-01   3.8308e-03
  8.5663e+00   7.6145e-01   4.5726e-03
  6.4736e+00   1.9010e-01   8.1704e-04


Result for 4-point Lagrange interpolation: 

In [86]:
print_out(x_test, y_interp_lagrange)

   max. error was:   4.1728e-06
      test_x     y_interp        error
  4.2715e-01   4.1428e-01   1.8644e-06
  1.9579e+00   9.2599e-01   1.9187e-06
  2.9450e+00   1.9532e-01   2.6011e-07
  6.2700e+00  -1.3187e-02  -5.4423e-07
  8.6223e-01   7.5930e-01   2.7952e-06
  1.4295e+00   9.9003e-01   4.1728e-06
  5.1583e+00  -9.0224e-01  -1.8862e-06
  6.8934e+00   5.7306e-01   3.3370e-06
  8.5663e+00   7.5688e-01   3.6755e-06
  6.4736e+00   1.8928e-01   2.2083e-08


Result for 4-point Hermit interpolation: 

In [87]:
print_out(x_test, y_interp_hermit)

   max. error was:   1.8789e-04
      test_x     y_interp        error
  4.2715e-01   4.1420e-01  -8.3681e-05
  1.9579e+00   9.2607e-01   8.1797e-05
  2.9450e+00   1.9535e-01   3.3899e-05
  6.2700e+00  -1.3172e-02   1.3671e-05
  8.6223e-01   7.5920e-01  -1.0013e-04
  1.4295e+00   9.9004e-01   1.2905e-05
  5.1583e+00  -9.0242e-01  -1.8789e-04
  6.8934e+00   5.7289e-01  -1.6625e-04
  8.5663e+00   7.5674e-01  -1.3634e-04
  6.4736e+00   1.8929e-01   5.9271e-06


### Explanation:
According to what we have seen in class, lets summarize the benefits and drawbacks of each type of interpolation: 

|           | Linear | 4-point Lagrange | 4-point Hermit |
| --------  | ------ | ---------------- | -------------- | 
| Benefits  | Simple and fast. Suitable for real-time applications where speed is crucial. | More accurate for smooth functions. Captures more detail of the data's shape. | High accuracy with smooth first derivatives. Suitable for smooth functions with known derivatives. |
| Drawbacks | Less accurate for non-linear functions. | Requires more points and is more computationally intensive. | Most complex and computationally intensive. |
| Error | $O(h^{2})$ | $O(h^{6})$ | $O(h^{5})$ |

According to data that we got, for the example data that was given, we get the least error using Lagrange interpolation - the expected result - because the linear is not accurate enough and Hermit is more suitable for function with known first derivative - in our case we calculated it artificially and used 3-point formula, so we got certain amount of error (possibly cumulative computational error) in the derivative.  We did get approximately the same order of magnitude of error in all of the methods. 



---

Lets compare the 3 interpolations for more sparse table:

In [88]:
y_interp_linear = np.zeros(ntests)
y_interp_lagrange = np.zeros(ntests)
y_interp_hermit = np.zeros(ntests)

for i in range(ntests):
    y_interp_linear[i]=interp(x[0:npoints:2],y[0:npoints:2],x_test[i],'Linear')
    y_interp_lagrange[i]=interp(x[0:npoints:2],y[0:npoints:2],x_test[i],'Lagrange4')
    y_interp_hermit[i]=interp(x[0:npoints:2],y[0:npoints:2],x_test[i],'Hermit4')

Result for linear interpolation: 

In [89]:
print_out(x_test, y_interp_linear)

   max. error was:   3.1216e-02
      test_x     y_interp        error
  4.2715e-01   4.1611e-01   1.8240e-03
  1.9579e+00   9.4847e-01   2.2483e-02
  2.9450e+00   2.0285e-01   7.5314e-03
  6.2700e+00  -1.4394e-02  -1.2081e-03
  8.6223e-01   7.6387e-01   4.5758e-03
  1.4295e+00   9.9541e-01   5.3869e-03
  5.1583e+00  -9.3345e-01  -3.1216e-02
  6.8934e+00   5.7886e-01   5.8088e-03
  8.5663e+00   7.8046e-01   2.3586e-02
  6.4736e+00   1.9014e-01   8.5728e-04


Result for 4-point Lagrange interpolation:

In [90]:
print_out(x_test, y_interp_lagrange)

   max. error was:   6.2757e-05
      test_x     y_interp        error
  4.2715e-01   4.1429e-01   1.1224e-05
  1.9579e+00   9.2605e-01   6.2757e-05
  2.9450e+00   1.9533e-01   1.2867e-05
  6.2700e+00  -1.3193e-02  -6.5504e-06
  8.6223e-01   7.5932e-01   2.6388e-05
  1.4295e+00   9.9006e-01   3.3862e-05
  5.1583e+00  -9.0226e-01  -2.9503e-05
  6.8934e+00   5.7308e-01   2.7025e-05
  8.5663e+00   7.5691e-01   3.6250e-05
  6.4736e+00   1.8929e-01   3.6740e-06


Result for 4-point Hermit interpolation: 

In [91]:
print_out(x_test, y_interp_hermit)

   max. error was:   8.4400e-04
      test_x     y_interp        error
  4.2715e-01   4.1401e-01  -2.6776e-04
  1.9579e+00   9.2606e-01   7.1185e-05
  2.9450e+00   1.9527e-01  -4.5293e-05
  6.2700e+00  -1.3133e-02   5.3595e-05
  8.6223e-01   7.5926e-01  -3.3452e-05
  1.4295e+00   9.8918e-01  -8.4400e-04
  5.1583e+00  -9.0235e-01  -1.1434e-04
  6.8934e+00   5.7291e-01  -1.4609e-04
  8.5663e+00   7.5685e-01  -2.3225e-05
  6.4736e+00   1.8908e-01  -1.9766e-04


### Explanation: 
We got that in most cases the error is one order of magnitude less than previously. Meaning we got bigger error - as expected from more sparce table, that has less values and bigger gaps between values. I used the "brute force" approach to calculate the index, so in bigger table we would get bigger computation time, but in this case it does not differ much. 

---

Also lets compare the results for even more sparce table (smaller by a factor of 4):

In [92]:
y_interp_linear = np.zeros(ntests)
y_interp_lagrange = np.zeros(ntests)
y_interp_hermit = np.zeros(ntests)

for i in range(ntests):
    y_interp_linear[i]=interp(x[0:npoints:4],y[0:npoints:4],x_test[i],'Linear')
    y_interp_lagrange[i]=interp(x[0:npoints:4],y[0:npoints:4],x_test[i],'Lagrange4')
    y_interp_hermit[i]=interp(x[0:npoints:4],y[0:npoints:4],x_test[i],'Hermit4')

Results for linear interpolation: 

In [93]:
print_out(x_test, y_interp_linear)

   max. error was:   1.3962e-01
      test_x     y_interp        error
  4.2715e-01   4.1693e-01   2.6493e-03
  1.9579e+00   1.0509e+00   1.2492e-01
  2.9450e+00   2.1054e-01   1.5228e-02
  6.2700e+00  -3.9765e-02  -2.6579e-02
  8.6223e-01   7.6717e-01   7.8781e-03
  1.4295e+00   1.0459e+00   5.5858e-02
  5.1583e+00  -1.0419e+00  -1.3962e-01
  6.8934e+00   5.8206e-01   9.0025e-03
  8.5663e+00   7.9280e-01   3.5920e-02
  6.4736e+00   1.8950e-01   2.1572e-04


Results for 4-point Lagrange interpolation:

In [94]:
print_out(x_test, y_interp_lagrange)

   max. error was:   8.5008e-04
      test_x     y_interp        error
  4.2715e-01   4.1414e-01  -1.3640e-04
  1.9579e+00   9.2666e-01   6.7596e-04
  2.9450e+00   1.9567e-01   3.4935e-04
  6.2700e+00  -1.3504e-02  -3.1747e-04
  8.6223e-01   7.5950e-01   1.9965e-04
  1.4295e+00   9.9088e-01   8.5008e-04
  5.1583e+00  -9.0249e-01  -2.5989e-04
  6.8934e+00   5.7324e-01   1.8300e-04
  8.5663e+00   7.5755e-01   6.7365e-04
  6.4736e+00   1.8927e-01  -1.5732e-05


Results for 4-point Hermit interpolation:

In [95]:
print_out(x_test, y_interp_hermit)

   max. error was:   1.2108e-03
      test_x     y_interp        error
  4.2715e-01   4.1326e-01  -1.0254e-03
  1.9579e+00   9.2520e-01  -7.8611e-04
  2.9450e+00   1.9627e-01   9.5867e-04
  6.2700e+00  -1.2543e-02   6.4272e-04
  8.6223e-01   7.5852e-01  -7.7275e-04
  1.4295e+00   9.9024e-01   2.1040e-04
  5.1583e+00  -9.0211e-01   1.1936e-04
  6.8934e+00   5.7214e-01  -9.1307e-04
  8.5663e+00   7.5714e-01   2.6842e-04
  6.4736e+00   1.8807e-01  -1.2108e-03


### Explanation:
We can see that, again, in each case the error has grown by about order of magnitude (and by about 2 orders of magnitude from the initial values) - this is predictable with the table that is 4 times more sparce than the initial one. 